# Introduction

This fifth notebook shows you how to explore the trust score results in a notebook. If you have not already done so, please run the [fourth notebook](patient-readmission-trust-scan.ipynb) to scan the models for their trust scores.

In this notebook, we will:
1. Load the previously saved trust score reports
2. Convert the results into a dataframe and display them


In [1]:
import numpy as np
import pandas as pd
from pprint import pprint

from certifai.scanner.report_reader import ScanReportReader
from certifai.scanner.report_utils import scores, construct_scores_dataframe
from certifai.scanner.builder import ExplanationType
from IPython.display import display, Markdown

# Loading the Trust Score Reports

To load the reports, we need to know the use case ID ('readmission') and the scan ID.

List the available use cases, and the scans within the 'readmission' use case.

In [2]:
reader = ScanReportReader("reports")
reader.list_usecases()
scans = reader.list_scans('readmission')
data=[[s['date'], ', '.join(s['reportTypes']), s['id']] for s in scans]
df = pd.DataFrame(data, columns=['date', 'evals', 'scan id']).sort_values(by=['date'], ascending=False)
print(df)

              date                                      evals       scan id
0  20201004T141144                              fairness, atx  98bbe86c99d0
1  20201004T131121  robustness, fairness, explainability, atx  e9ffd5d26bb7
2  20201004T122043                                explanation  5c54861f2e8a
3  20201004T105121                                explanation  700a8c0e9c86
4  20201004T104721                                explanation  4ed8ce9304b8
5  20201004T104504                                explanation  8cfbc13c5900
6  20201004T104419                                explanation  863495d8b85c
7  20201004T101611                                explanation  794cc03f0b4d


In [3]:
latest_scan = df[df['evals'].str.contains('fairness') & df['evals'].str.contains('robustness')].iloc[0]
results = reader.load_scan('readmission', latest_scan['scan id'])

# Extract the results

In this section we'll construct dataframes containing the score results and display them.

Print out the fairness results.

In [4]:
df = construct_scores_dataframe(scores('fairness', results), include_confidence=False)
age_cols = [str(col) for col in df.columns if '5' in col]
display(df)

,context,type,overall fairness,Feature (age),Group details (15),Group details (25),Group details (35),Group details (45),Group details (5),Group details (55),...,Feature (gender),Group details (Female),Group details (Male),Feature (race),Group details (AfricanAmerican),Group details (Asian),Group details (Caucasian),Group details (Hispanic),Group details (Other),Group details (nan)
logit (burden),logit,burden,56.175510,56.237860,0.026482,0.234150,0.093466,0.124085,0.000493,0.055415,...,84.721827,0.088344,0.065051,64.198991,0.079880,0.024047,0.096890,0.065421,0.065434,0.014086
mlp (burden),mlp,burden,71.895673,77.140629,0.043008,0.107225,0.068858,0.074807,0.010121,0.067391,...,88.830662,0.074335,0.059524,71.926641,0.064545,0.034484,0.078982,0.082273,0.059485,0.018051


For an example of visualizing the fairness results in a chart, see the [fairness metrics notebook](https://raw.githubusercontent.com/CognitiveScale/cortex-certifai-examples/sdk-model-examples/notebooks/fairness_metrics).

Print out the robustness scores.

In [5]:
df = construct_scores_dataframe(scores('robustness', results), include_confidence=True)
display(df)

,context,robustness,robustness lower bound,robustness upper bound
mlp,mlp,47.700394,42.569500,52.792079
logit,logit,52.326452,49.128134,55.520053


Print out the explainability scores.

In [6]:
df = construct_scores_dataframe(scores('explainability', results), include_confidence=False)
display(df)

,context,explainability,Num features (1),Num features (10),Num features (2),Num features (3),Num features (4),Num features (5),Num features (6),Num features (7),Num features (8),Num features (9)
logit,logit,94.726562,53.1250,0.0,42.96875,3.90625,0.0,0.0,0.0,0.0,0.0,0.0
mlp,mlp,96.640625,73.4375,0.0,21.87500,4.68750,0.0,0.0,0.0,0.0,0.0,0.0
